In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from utils import read_data
from gensim.models import Word2Vec
import time
from tqdm import tqdm
import concurrent.futures

In [2]:
model = Word2Vec.load('data/word2vec.model')

In [20]:
word_list = model.wv.key_to_index.keys()
type(word_list)

dict_keys

In [24]:
if '1005' in model.wv.key_to_index:
    print(True)
else:
    print(False)

True


In [74]:
products.loc['1002']

product_name    All Natural Stevia Liquid Extract Sweetener
aisle                                    baking ingredients
department                                           pantry
Name: 1002, dtype: object

In [77]:
products.loc[['1002', '1003']].department.values

array(['pantry', 'pantry'], dtype=object)

In [81]:
def get_product_names(dic, pid):
    products=pd.read_pickle('data/products.pickle')[['product_id', 'product_name', 'aisle', 'department']]
    products['product_id'] = products['product_id'].astype('str')
    print(products.loc[products['product_id']==pid])
    return pd.DataFrame(dic, columns=['product_id', 'similarity']).merge(products)
pid = 1003
dic = model.wv.most_similar(pid, topn=len(model.wv))
print(len(dic))
get_product_names(dic, pid)

49582
Empty DataFrame
Columns: [product_id, product_name, aisle, department]
Index: []


,product_id,similarity,product_name,aisle,department
0,30639,0.726292,Organic Orange Juice,juice nectars,beverages
1,17131,0.663997,Cold-Pressed Organic Apple,refrigerated,beverages
2,5589,0.649040,Tangerine Fruit Juice,refrigerated,beverages
3,27729,0.640782,Cold-Pressed Organic Orange,refrigerated,beverages
4,16387,0.610981,Cold-Pressed Organic Grapefruit,juice nectars,beverages
...,...,...,...,...,...
49577,31486,-0.496597,Zero Calorie Sweetener,baking ingredients,pantry
49578,11773,-0.498103,Classic Roast Medium Ground Coffee,coffee,beverages
49579,13072,-0.502906,Bay Scallops,seafood counter,meat seafood
49580,6581,-0.506150,Cucumber,fresh vegetables,produce


In [42]:
dic

[('44031', 0.7960940003395081),
 ('8013', 0.7890138030052185),
 ('130', 0.7789651155471802),
 ('48812', 0.7788556814193726),
 ('21572', 0.7785413861274719)]

In [30]:
products=pd.read_pickle('data/products.pickle')[['product_id', 'product_name', 'aisle', 'department']].set_index('product_id')
products.index = products.index.astype('str')

In [31]:
vocab_len = len(model.wv)
vocab_len

49583

In [60]:
lst = {'1003':products.loc['1003', 'department']}
print(lst)
for k, v in lst.items():
    print(k, v)

{'1003': 'pantry'}
1003 pantry


In [82]:
start_time = time.time()

threshold = 0.5
res = []
for pid, department in tqdm({'1003':products.loc['1003', 'department']}.items()):
#     print(pid, department)
    for pair in model.wv.most_similar(pid, topn=len(model.wv)):
        sub_pid, sub_sim_score = pair[0], pair[1]
        sub_department = products.at[sub_pid, 'department']
        
        if sub_department == department and sub_sim_score > threshold:
            res.append([pid, sub_pid, sub_sim_score])
            
end_time = time.time()
print('code using {:.2f} mins'.format((end_time - start_time) / 60))

100%|█████████████████████████████████████████████| 1/1 [00:00<00:00,  2.49it/s]

code using 0.01 mins


In [83]:
len(res)

47

In [70]:
47 + 197

244

In [106]:
type(products['department'].head().reset_index().values)

numpy.ndarray

In [108]:
products.index.values

array(['1', '2', '3', ..., '49686', '49687', '49688'], dtype=object)

In [105]:
threshold = 0.5
res = []
def multi(pid, department):
    for pair in model.wv.most_similar(pid, topn=len(model.wv)):
        sub_pid, sub_sim_score = pair[0], pair[1]
        sub_department = products.at[sub_pid, 'department']
        
        if sub_department == department and sub_sim_score > threshold:
            return [pid, sub_pid, sub_sim_score]
if __name__ == '__main__':          
    with concurrent.futures.ProcessPoolExecutor() as executor:
        results = executor.map(multi, products['department'].head().reset_index().values)
    print(type(results))
    res = [result for result in results]

<class 'generator'>


Process SpawnProcess-7:
Process SpawnProcess-6:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/qiaowang/opt/anaconda3/envs/Learning/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/Users/qiaowang/opt/anaconda3/envs/Learning/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/Users/qiaowang/opt/anaconda3/envs/Learning/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/qiaowang/opt/anaconda3/envs/Learning/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/qiaowang/opt/anaconda3/envs/Learning/lib/python3.9/concurrent/futures/process.py", line 237, in _process_worker
    call_item = call_queue.get(block=True)
  File "/Users/qiaowang/opt/anaconda3/envs/Learning/lib/python3.9/concurrent/futures/process.py", line 237, in _process_worker
 

BrokenProcessPool: A process in the process pool was terminated abruptly while the future was running or pending.

In [93]:
len(res)

2257

In [115]:
res_df = pd.DataFrame(res, columns=['product_id', 'subsitute_id', 'similarity_score'])
res_df.loc[res_df.product_id=='1'].shape

(452, 3)

In [98]:
products.loc['21572', ]

product_name    Cheez-It Cheddar Cracker
aisle                           crackers
department                        snacks
Name: 21572, dtype: object

In [31]:
products['department'].reset_index().head().values

array([[1, 'snacks'],
       [2, 'pantry'],
       [3, 'beverages'],
       [4, 'frozen'],
       [5, 'pantry']], dtype=object)